# RAG Metrics

The list below is some of the metrics that are used to measure the performance of a RAG model.
As the peformance depends on 3 difference components: Answer Synthesis (LLM Parameters and Prompt), Retreival (Search parameters and ranking), and Indexing (Embedding model and chunking methods), each of the metrics attempt at measuring different components. The list below is not exhaustive and there are different metrics for different use cases, for example, measuring the tone of the final answer for customer facing models.

**Correctness**: 

Whether the generated answer matches that of the reference answer given the query (requires labels).

**Semantic Similarity**: 

Whether the predicted answer is semantically similar to the reference answer (requires labels).

**Faithfulness**: 

Evaluates if the answer is faithful to the retrieved contexts (in other words, whether if there's hallucination).

**Context Relevancy**: 

Whether retrieved context is relevant to the query. 

**Answer Relevancy**: 

Whether the generated answer is relevant to the query. 

**Guideline Adherence**: 

Whether the predicted answer adheres to specific guidelines.

**Hit Rate**:

Hit rate calculates the fraction of queries where the correct answer is found within the top-k retrieved documents. In simpler terms, it’s about how often our system gets it right within the top few guesses.

**Mean Reciprocal Rank (MRR)**:

MRR serves as a metric for evaluating the accuracy of a system by examining the rank of the highest-placed relevant document for each query. It calculates the average of the reciprocals of these ranks across all queries. For instance, if the first relevant document is ranked highest, the reciprocal rank is 1; if it’s second, the reciprocal rank is 1/2, and so forth.

# Preparation

Run the cell below and restart the kernel. Once the kernel restarts, skip to **Set Up**

In [ ]:
!pip install -q -U llama-index llama-index-llms-ibm pydantic
print("Done Installing!")

# Set Up

Before we measure the performances, we need a way to tell WatsonX.AI that it's an authroized user trying to use the models. Follow the instruction below to retrieve your API Key.

Instruction:

When done, paste **your api key below** and run both cells

In [ ]:
myApikey="Paste your API key here"

In [ ]:

from llama_index.core.evaluation import BatchEvalRunner
from llama_index.core.evaluation import AnswerRelevancyEvaluator
from llama_index.core.evaluation import CorrectnessEvaluator
from llama_index.llms.ibm import WatsonxLLM
from statistics import mean
import os
import nest_asyncio
import asyncio
import warnings
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

nest_asyncio.apply()

watsonx_llm = WatsonxLLM(
    model_id="mistralai/mistral-large", # granite or mistral 
    url="https://us-south.ml.cloud.ibm.com",
    apikey=myApikey,
    project_id=os.getenv("PROJECT_ID"),
    max_new_tokens=1000
)

In [ ]:
async def evaluate_rag(log,ground_truth=None):

    # The metrics to be computed is: Answer Relevancy
    pattern = r'<|.*?\|>'
    parsed_list=[x.replace("\n","") for x in re.split(pattern, log) if len(x)>2]
    queries=parsed_list[1::2] 
    responses=parsed_list[2::2] 
    
    ar_eval=AnswerRelevancyEvaluator(llm=watsonx_llm)

    ar_runner = BatchEvalRunner(
        {"answer_relevancy": ar_eval},workers=5,show_progress=True)

    eval_results_ar = await ar_runner.aevaluate_response_strs(queries=queries, response_strs=responses)
    ar_score=mean([float(eval_results_ar['answer_relevancy'][x].score) for x in range(len(queries))])*100
    
    results=[eval_results_ar['answer_relevancy'][x].feedback for x in range(len(queries))]
    feedback, scores = map(list, zip(*(s.split("\n\n") for s in results)))
    feedback=[x.replace("\n","") for x in feedback]
    print("\n")
    
    print(f"The average Answer Relevancy score is {round(ar_score,2)}%")
    print("\n")

    df=pd.DataFrame(data={'Query':queries,'Response':responses,'Feedback':feedback,'Score (max:2)':scores})
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)
    



    

    

With the LLM connection establsihed, copy paste your RAG log from your Prompt Lab session and run both cells.

The responses will be evaluated against the following two criteria: <br> 1. Does the provided response match the subject matter of the user's query? <br> 2. Does the provided response attempt to address the focus or perspective on the subject matter taken on by the user's query? <br> Each question above is worth 1 point.

In [ ]:
myLog="""Copy-paste your log here
"""

In [ ]:
await evaluate_rag(myLog)